In [1]:
# coding: utf-8

'''
Created on 21 мая 2016 г.

@author: keen
'''

from sklearn import datasets
import pickle

from scipy.sparse import csr_matrix
from sklearn import preprocessing
from numpy import ndarray
from numpy import asarray
from numpy.random import randint as rint
import datetime
from random import randint
import numpy
import CO2_tree_reg as co2t
import CO2_forest_reg as co2f
from sklearn.metrics import accuracy_score
import os

In [ ]:


x_train = []
Y_train = []
for ds in os.listdir("processed/train"):
    ds_name = os.path.join("processed/train",ds)
    data = None
    labels = None
    for f in os.listdir(ds_name):
        filename = os.path.join(ds_name,f)  
        if filename.find("data") > -1:
            data = numpy.nan_to_num(numpy.load(filename),posinf=0,neginf=0)
        else:                   
            labels = numpy.nan_to_num(numpy.load(filename),posinf=0,neginf=0)
            print (filename,labels.mean())
    if data is not None:     
        x_train.append(data.astype(numpy.float64)[:1000])
        Y_train.append(labels.mean(axis=1).mean(axis=1).astype(float).reshape(-1,1)[:1000])
        
x_validate = []
Y_validate = []
for ds in os.listdir("processed/outumn"):
    ds_name = os.path.join("processed/outumn",ds)
    data = None
    labels = None
    for f in os.listdir(ds_name):
        filename = os.path.join(ds_name,f)  
        if filename.find("data") > -1:
            data = numpy.nan_to_num(numpy.load(filename),posinf=0,neginf=0)
        else:                   
            labels = numpy.nan_to_num(numpy.load(filename),posinf=0,neginf=0)
            print (filename,labels.mean())
    if data is not None:        
        x_validate.append(data.astype(numpy.float64)[:1000])
        Y_validate.append(labels.mean(axis=1).mean(axis=1).astype(float).reshape(-1,1)[:1000])

data={}
data['NVDI'] = [[numpy.vstack(x_train),numpy.vstack(Y_train)],[numpy.vstack(x_validate),numpy.vstack(Y_validate)],(228,228,3)]        

In [2]:
data={}
data['NVDI'] = [[[],[]],[[],[]],(228,228,3)]

In [3]:
TRAIN_SIZE = 1000
TEST_SIZE = 1000
SHAKE = True
NJOBS = 15

In [4]:
from deepforest import CascadeForestRegressor
from sklearn.ensemble import RandomForestRegressor

TREE_D = [4]
TREE_F = [0.02]


KERNEL = ['linear']
KERNEL_PRUNE = [True,False]
KERNEL_PRUNE_C = [1]
KERNEL_PRUNE_N = [0.1]

n_estimators = 2

estimators=[]

for t in TREE_D:
    for f in TREE_F:
        tmp = []
        tmp.append('RF')
        tmp.append(t)
        tmp.append(f)
        tmp.append([RandomForestRegressor(max_depth=t,max_features=f,random_state=i,n_jobs=NJOBS) for i in range(n_estimators)])
        estimators.append(tmp)  
        
        for k in KERNEL:
            for pr in KERNEL_PRUNE:
                if pr:
                    for c in KERNEL_PRUNE_C:
                        for n in KERNEL_PRUNE_N:
                            tmp = []
                            tmp.append('KF')
                            tmp.append(t)
                            tmp.append(f)
                            tmp.append(k)
                            tmp.append("Pruned")
                            tmp.append(c)
                            tmp.append(n)
                            tmp.append([co2f.CO2_forestReg(C=5500, min_samples_leaf=4,dual=False,tol = 0.0001,max_iter=10000,kernel=k,max_deth=t,n_jobs=NJOBS,sample_ratio=1.0, feature_ratio = f,n_estimators=30,gamma=100,dropout_low=0.0,dropout_high=1.0,noise=0.,cov_dr=0.,reinforced = True, prune_level = n,reC=c) for i in range(n_estimators)])                            
                            estimators.append(tmp) 
                    else:
                        tmp = []

                        tmp.append('KF')
                        tmp.append(t)
                        tmp.append(f)
                        tmp.append(k)
                        tmp.append("Not pruned") 
                        tmp.append([co2f.CO2_forestReg(C=5500, min_samples_leaf=4,dual=False,tol = 0.0001,max_iter=10000,kernel=k,max_deth=t,n_jobs=NJOBS,sample_ratio=1.0, feature_ratio = f,n_estimators=30,gamma=100,dropout_low=0.0,dropout_high=1.0,noise=0.,cov_dr=0.) for i in range(n_estimators)])                            

                        estimators.append(tmp)  
                    
WIN = [False]

DS_WIN_SZ = {
    'NVDI':[32],
}


In [ ]:
from os.path import exists 

from sklearn.metrics import mean_squared_error

def LOG(str_):
    with open("log_nvdi_sp_out.txt", "a") as f:
        if str_ != '\n':
            f.write(str(str_) + ";")
        else:    
            f.write(str_)
            
def INLOG(str_):
    if exists("log_nvdi_sp_out.txt"):
        with open("log_nvdi_sp_out.txt",'r') as f:
            log = f.read()
        return log.find(str_) > -1
    return False            
            
for d in data:
    x_train = data[d][0][0]
    Y_train = data[d][0][1]
    x_validate = data[d][1][0]
    Y_validate = data[d][1][1]
    if SHAKE:
        if exists(d + "_train_data.npy"): 
            x_train = numpy.load(d + "_train_data.npy")
            Y_train = numpy.load(d + "_train_labels.npy")
            x_validate_sp = numpy.load(d + "_test_data_sp.npy")
            Y_validate_sp = numpy.load(d + "_test_labels_sp.npy")
            x_validate_out = numpy.load(d + "_test_data_out.npy")
            Y_validate_out = numpy.load(d + "_test_labels_out.npy")
            
        else:         
            tr_idxs = numpy.random.permutation(x_train.shape[0])[:TRAIN_SIZE]         
            tst_idxs = numpy.random.permutation(x_validate.shape[0])[:TEST_SIZE]         
            x_train = x_train[tr_idxs]
            Y_train = Y_train[tr_idxs]
            x_validate = x_validate[tst_idxs]
            Y_validate = Y_validate[tst_idxs]

            numpy.save(d + "_train_data",x_train)
            numpy.save(d + "_train_labels",Y_train)
            numpy.save(d + "_test_data_out",x_validate)
            numpy.save(d + "_test_labels_out",Y_validate)
            break
        
    if data[d][2][2] > 1:
        x_train = x_train.reshape((data[d][2][2],-1,data[d][2][0],data[d][2][1]))
        x_validate_sp = x_validate_sp.reshape((data[d][2][2],-1,data[d][2][0],data[d][2][1]))
        x_validate_out = x_validate_out.reshape((data[d][2][2],-1,data[d][2][0],data[d][2][1]))        
    else:
        x_train = x_train.reshape((-1,data[d][2][0],data[d][2][1]))
        x_validate_sp = x_validate_sp.reshape((-1,data[d][2][0],data[d][2][1]))
        x_validate_out = x_validate_out.reshape((-1,data[d][2][0],data[d][2][1]))
        
    for w in WIN:
        if w:
            wss_ = DS_WIN_SZ[d]
        else:
            wss_ = [1]
        for wss in wss_:
            for e_idx,e in enumerate(estimators):
                est = e[len(e) - 1]
                
                l = d + ";"
                l += str(w) + ";"
                l += str(wss) + ";" 
                for i in range(len(e) - 1):
                    l += str(e[i]) + ";" 

                if not INLOG(l):                
                    model = CascadeForestRegressor(max_layers=3)
                    model.set_estimator(est)                    
                    if w and data[d][2][0] < wss:
                        model.set_win([wss,-1,-1],[4,1,1],[data[d][2][0],1,1],[data[d][2][1],1,1],[0,0,0],[2,0,0],[False,True,True])
                        model.fit(x_train, Y_train)#
                        pred_sp = model.predict(x_validate_sp)
                        pred_out = model.predict(x_validate_out)
                    else: 
                        model.fit(x_train.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2]), Y_train)#
                        pred_sp = model.predict(x_validate_sp.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2]))
                        pred_out = model.predict(x_validate_out.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2]))
                        

                    LOG(d)
                    LOG(str(w))
                    LOG(str(wss))
                    for i in range(len(e) - 1):
                        LOG(e[i])                    
                    print (d,w,wss,e[:len(e)-1])


                    LOG(mean_squared_error(Y_validate_sp, pred_sp))
                    LOG(mean_squared_error(Y_validate_out, pred_out))
                    #print(mean_squared_error(Y_validate, pred))
                    LOG('\n')
                else:
                    print ("Skip:",d,w,wss,e[:len(e)-1])                    


/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[2022-08-24 18:47:46.160] Start to fit the model:
[2022-08-24 18:47:46.160] Fitting cascade layer = 0 
[2022-08-24 18:48:26.147] layer = 0  | Val MSE = 0.00896 | Elapsed = 39.987 s
[2022-08-24 18:48:26.630] Fitting cascade layer = 1 
[2022-08-24 18:49:06.760] layer = 1  | Val MSE = 0.00877 | Elapsed = 40.131 s
[2022-08-24 18:49:07.254] Fitting cascade layer = 2 
[2022-08-24 18:49:47.325] layer = 2  | Val MSE = 0.00883 | Elapsed = 40.071 s
[2022-08-24 18:49:47.325] Early stopping counter: 1 out of 2
[2022-08-24 18:49:47.325] Reaching the maximum number of layers: 3
[2022-08-24 18:49:47.473] Start to evalute the model:
[2022-08-24 18:49:47.955] Evaluating cascade layer = 0 
[2022-08-24 18:49:50.587] Evaluating cascade layer = 1 
[2022-08-24 18:49:53.289] Evaluating cascade layer = 2 
[2022-08-24 18:49:56.077] Start to evalute the model:
[2022-08-24 18:49:56.499] Evaluating cascade layer = 0 
[2022-08-24 18:49:59.150] Evaluating cascade layer = 1 
[2022-08-24 18:50:01.850] Evaluating casc

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[2022-08-24 20:50:08.695] layer = 0  | Val MSE = 0.00499 | Elapsed = 7203.949 s
[2022-08-24 20:50:09.159] Fitting cascade layer = 1 


In [ ]:
x_train

In [ ]:
m=co2t.CO2TreeReg(C=5500, min_samples_leaf=4,dual=False,tol = 0.0001,max_iter=10000,kernel="polynomial",max_deth=3,sample_ratio=1.0, feature_ratio = 0.1,gamma=100,dropout_low=0.0,dropout_high=1.0,noise=0.,cov_dr=0.)                      

In [ ]:
model=m

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
model.fit(csr_matrix(x_train.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2])),Y_train)#

In [ ]:
model.predict(csr_matrix(x_validate.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2])))

In [ ]:
pred

In [ ]:
for n in model.layers_['layer_0'].estimators_['0-0-custom'].estimators_[0].trees[0].nodes:
    print(n.p0,n.p1)

In [ ]:
for n in model.nodes:
    print(n.p0,n.p1)   

In [ ]:
model.leaves

In [ ]:
for n in model.layers_['layer_0'].estimators_['0-0-custom'].estimators_[0].trees[1].nodes:
    print(n.p0,n.p1)

In [ ]:
model.layers_['layer_0'].estimators_['0-0-custom'].estimators_[0].trees[0].leaves

In [ ]:
model.layers_['layer_0'].estimators_['0-0-custom'].estimators_[0].predict(x_validate.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2]))

In [ ]:
model.layers_['layer_0'].estimators_['0-0-custom'].estimators_[0].trees[0].predict(csr_matrix(x_validate.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2])))

In [ ]:
model.layers_['layer_0'].estimators_['0-0-custom'].estimators_[0].trees[0].nodes[9].stamp_sign(csr_matrix(x_validate.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2])))

In [ ]:
x_validate

In [ ]:
x_train

In [ ]:
Y_validate

In [ ]:
numpy.unique(model.layers_['layer_0'].estimators_['0-0-custom'].estimators_[0].trees[0].nodes[0].stamp_sign(csr_matrix(x_train.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2]))),return_counts=True)

In [ ]:
from sklearn.cluster import KMeans
k = KMeans(n_clusters=2)
k.fit_predict(Y_train.reshape(-1,1))*2 - 1  

In [ ]:
numpy.unique(model.nodes[0].stamp_sign(csr_matrix(x_train.reshape(-1,data[d][2][0]*data[d][2][1]*data[d][2][2]))),return_counts=True)